In [1]:
import cupy as cp
import scanpy as sc
import pandas as pd
import numpy as np

In [2]:
def silhouette_samples_cuda(X, labels, device_id=0, chunk_size=1000):
    # 将数据转移到指定的GPU上，使用float32以减少显存
    with cp.cuda.Device(device_id):
        X_gpu = cp.asarray(X, dtype=cp.float32)
        labels_gpu = cp.asarray(labels)

        # 获取数据点的数量
        n_samples = X.shape[0]
        
        # 初始化 a(i) 和 b(i)
        a = cp.zeros(n_samples, dtype=cp.float32)
        b = cp.zeros(n_samples, dtype=cp.float32)

        # 获取所有唯一的标签
        unique_labels = cp.unique(labels_gpu)

        # 分块计算距离矩阵
        for i in range(0, n_samples, chunk_size):
            end_i = min(i + chunk_size, n_samples)

            # 计算块内的距离矩阵
            distance_matrix_chunk = cp.linalg.norm(X_gpu[i:end_i, None] - X_gpu, axis=2)

            # 逐个计算每个样本点的 a(i) 和 b(i)
            for j in range(end_i - i):
                current_label = labels_gpu[i + j]
                
                # 计算 a(i): 簇内距离平均值
                same_cluster_mask = (labels_gpu == current_label)
                same_cluster_distances = distance_matrix_chunk[j, same_cluster_mask]
                if cp.sum(same_cluster_mask) > 1:
                    a[i + j] = cp.sum(same_cluster_distances) / (cp.sum(same_cluster_mask) - 1)
                else:
                    a[i + j] = 0

                # 计算 b(i): 最近簇距离平均值
                b[i + j] = cp.inf
                for label in unique_labels:
                    if label != current_label:
                        other_cluster_mask = (labels_gpu == label)
                        other_cluster_distances = distance_matrix_chunk[j, other_cluster_mask]
                        if other_cluster_distances.size > 0:
                            b[i + j] = cp.minimum(b[i + j], cp.mean(other_cluster_distances))

        # 计算轮廓系数 s(i)
        s = (b - a) / cp.maximum(a, b)
        return s

def F1_silhouette(embeds, cell_type, omics, device_id=0, chunk_size=1000):
    # 将cell_type和omics转换为分类数据并转换为整数编码
    cell_type_encoded = pd.Categorical(cell_type).codes
    omics_encoded = pd.Categorical(omics).codes

    # 使用指定的GPU设备进行计算
    with cp.cuda.Device(device_id):
        # 确保数据在GPU上
        embeds_gpu = cp.asarray(embeds, dtype=cp.float32)
        cell_type_gpu = cp.asarray(cell_type_encoded)
        omics_gpu = cp.asarray(omics_encoded)
        
        # 计算每种标记的轮廓系数，使用分块
        s_ct = silhouette_samples_cuda(embeds_gpu, cell_type_gpu, device_id, chunk_size)
        s_omics = silhouette_samples_cuda(embeds_gpu, omics_gpu, device_id, chunk_size)
        
        # 归一化轮廓系数
        s_ct1 = (s_ct + 1) / 2
        s_omics1 = (s_omics + 1) / 2
        
        # 计算F1_silhouette
        F1_sil = 2 * ((1 - s_omics1) * s_ct1) / (1 - s_omics1 + s_ct1)
    return F1_sil

In [4]:
datasets = ['genes_PBMC10k75', 'genes_PBMC10k50', 'genes_PBMC10k25', 'PBMC10k', 'peaks_PBMC10k25', 'peaks_PBMC10k50', 'peaks_PBMC10k75']
methods = ['scMamba2']
for method in methods:
    F1_sil_all = []
    for dataset in datasets:
        path = f"../results/robustness_analysis/{dataset}batchsize256projection_dim50"
        adata = sc.read_h5ad(f"{path}/concat.h5ad")
        # 指定GPU ID，例如将计算放在设备ID为1的GPU上
        device_id = 6
        chunk_size = 100000  # 设置分块大小，视显存情况调整
        F1_sil_cuda = F1_silhouette(
            adata.obsm['X_umap'], 
            adata.obs['cell_type'].values, 
            adata.obs['modality'].values, 
            device_id, 
            chunk_size
        )
        # 将结果从GPU转移到CPU进行打印
        F1_sil_cuda_cpu = cp.asnumpy(F1_sil_cuda)
        F1_sil_all.append(F1_sil_cuda_cpu)
    F1_sil_all = np.column_stack(F1_sil_all)
    np.save(f"{method}_F1_sil_PBMC10k.npy", F1_sil_all)

In [ ]:
datasets = ['AD', 'AD25', 'AD50', 'AD75']
methods = ['scCLIP', 'GLUE', 'CVQVAE']
for method in methods:
    F1_sil_all = []
    for dataset in datasets:
        path = f"{method}/{dataset}"
        adata = sc.read_h5ad(f"{path}/concat.h5ad")
        # 指定GPU ID，例如将计算放在设备ID为1的GPU上
        device_id = 1
        chunk_size = 100  # 设置分块大小，视显存情况调整
        F1_sil_cuda = F1_silhouette(
            adata.obsm['X_umap'], 
            adata.obs['cell_type'].values, 
            adata.obs['modality'].values, 
            device_id, 
            chunk_size
        )
        # 将结果从GPU转移到CPU进行打印
        F1_sil_cuda_cpu = cp.asnumpy(F1_sil_cuda)
        F1_sil_all.append(F1_sil_cuda_cpu)
    F1_sil_all = np.column_stack(F1_sil_all)
    np.save(f"{method}_F1_sil.npy", F1_sil_all)

In [ ]:
datasets = ['genes_AD75', 'genes_AD50', 'genes_AD25', 'AD', 'peaks_AD25', 'peaks_AD50', 'peaks_AD75']
methods = ['scMamba']
for method in methods:
    F1_sil_all = []
    for dataset in datasets:
        path = f"{method}/{dataset}"
        adata = sc.read_h5ad(f"{path}/concat.h5ad")
        # 指定GPU ID，例如将计算放在设备ID为1的GPU上
        device_id = 4
        chunk_size = 100  # 设置分块大小，视显存情况调整
        F1_sil_cuda = F1_silhouette(
            adata.obsm['X_umap'], 
            adata.obs['cell_type'].values, 
            adata.obs['modality'].values, 
            device_id, 
            chunk_size
        )
        # 将结果从GPU转移到CPU进行打印
        F1_sil_cuda_cpu = cp.asnumpy(F1_sil_cuda)
        F1_sil_all.append(F1_sil_cuda_cpu)
    F1_sil_all = np.column_stack(F1_sil_all)
    np.save(f"{method}_F1_sil.npy", F1_sil_all)

In [ ]:
for _ in F1_sil_all:
    print(_.shape)

In [3]:
# 加载数据
dataset = "AD"
out_dir = f"scCLIP/{dataset}"
adata = sc.read_h5ad(f"{out_dir}/concat.h5ad")

In [ ]:
# 指定GPU ID，例如将计算放在设备ID为1的GPU上
device_id = 1
chunk_size = 100  # 设置分块大小，视显存情况调整
F1_sil_cuda = F1_silhouette(adata.X, adata.obs['cell_type'].values, adata.obs['modality'].values, device_id, chunk_size)

# 将结果从GPU转移到CPU进行打印
F1_sil_cuda_cpu = cp.asnumpy(F1_sil_cuda)
print(F1_sil_cuda_cpu)

In [ ]:
F1_sil_cpu = np.load("F1_sil_cpu.npy")
F1_sil_cpu

In [ ]:
F1_sil_cuda = np.load("F1_sil_cuda.npy")
F1_sil_cuda

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


# 准备数据框架
data = {'Values': np.concatenate([F1_sil_cuda, F1_sil_cpu]),
        'Category': ['Data 1'] * len(F1_sil_cuda) + ['Data 2'] * len(F1_sil_cpu)}

# 使用 Seaborn 绘制箱线图
sns.boxplot(x='Category', y='Values', data=data)

# 设置标题和标签
plt.title('Boxplot of Two Datasets with Seaborn')
plt.ylabel('Values')

# 显示图形
plt.show()
